In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import torch
import torchvision
import numpy as np

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Load data

In [4]:
train_data = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
val_data = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=torchvision.transforms.ToTensor())

In [5]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=len(train_data))
x, y = next(iter(train_loader))

h, w = x.shape[2], x.shape[2]

means = torch.zeros(len(train_data.classes), h, w)
vars = torch.zeros(len(train_data.classes), h, w)

for digit in range(len(train_data.classes)):
    subset = x[torch.where(y == 0)[0], :, :]
    means[digit, :, :] = torch.mean(subset, dim=0)
    vars[digit, :, :] = torch.var(subset, dim=0)

means, vars = means.to(device), vars.to(device)

In [6]:
batch_size = 64

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)

## I try out with unit Gaussian here. Need to consider all these 10 distributions.

In [22]:
from model import RealNVP

mask = np.arange(h * w / 2) * 2  # 0, 2, 4, ...
base_dist = torch.distributions.multivariate_normal.MultivariateNormal(torch.zeros(28 * 28).to(device),
                                              torch.eye(28 * 28).to(device))
model = RealNVP(mask, base_dist).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

#TODO: seems log(p_x) and log_det_inv_j would be large negative number. e.g. -1e04

In [ ]:
epochs = 20
print_every = 1

for epoch in range(epochs):
    total_loss = 0
    for data in train_loader:
        x, y = data
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        loss = - torch.sum(model.log_prob(x)) / x.shape[0]
        loss.backward()
        optimizer.step()
        total_loss += loss
    total_loss /= len(train_loader)

    if epoch % print_every == 0:
        print(f"Epoch {epoch} -> loss: {total_loss.item():.2f}")